El archivo homicidios viene con dos hojas de excel que son victimas y hechos, abro cada una en un notebook diferente para modificar y tranformar los valores que se necesite

La información que contiene la hoja de datos hechos es hechos de transito con vehículos en movimiento 

In [ ]:
pip install geopy

In [1]:
import json
import pandas as pd
import ast
from datetime import datetime, time
from geopy.geocoders import Nominatim

In [2]:
# Abro libro hechos del archivo homicidios y lo dejo como dataframe de csv

hechos_original = r"C:\Users\Usuario\Henry\PI2_DA\Consigna\hechos_original.csv"


In [3]:
df_hechos = pd.read_csv(hechos_original)

In [ ]:
df_hechos.info()

In [ ]:
df_hechos.head()

Se puede ver que este conjunto de datos cuenta con 21 columnas reterentes a las caracteristicas del evento como la fecha el numero de victimas, el lugar, y la ubicación que se podría normalizar de mejor forma. inicialmente voy a dejar una columna con la fecha y la hora para borrar la columnas que contienen esta información

In [6]:
# Borro columna HH que ya esta en hora

df_hechos = df_hechos.drop(columns=['HH'])


In [ ]:
df_hechos.info()

Creo función para verificar datos
Funcion verificar_tipos_datos

La función verificar_tipo_datos toma un DataFrame de Pandas code argumento y realiza un análisis detallado 
de cada columna. Genera  el nombre de la columna, el tipo de datos, 
el porcentaje de valores no nulos, el porcentaje de valores nulos y la cantidad de valores 
nulos para cada columna. Muestra las columnas con valores nulos ordenadas de mayor a menor según el porcentaje de valores nulos.
El resultado es un DataFrame que proporciona una visión detallada de la calidad de los datos en el 
DataFrame original.

In [11]:
df_hechos.size
# es corto

13920

In [225]:
# De entrada borro na
# df_hechos.dropna()
# No puedo borrar los na por que me borra todo, debe haber una columna que tiene en todas las casillas ne

In [12]:
# Borro duplicados que si puedo borrar

df_hechos = df_hechos.drop_duplicates()

In [13]:
df_hechos.size
# No hay duplicados pero no sobra

13920

In [14]:
# Me aseguro que toda la columna de fecha tenga el mismo formato YY-MM-DD 

df_hechos['FECHA'] = pd.to_datetime(df_hechos['FECHA'], format='%Y-%m-%d')


In [ ]:
df_hechos.info()

In [16]:
# Dado que esa columna ya tiene la info de toda la fecha borro columnas de año, mes y dia

columnas_borrar = ['AAAA', 'MM', 'DD']

df_hechos = df_hechos.drop(columns = columnas_borrar, axis=1)


In [ ]:
df_hechos.info()

Ya me ocupe de la hora, ahora revisaré todas las columnas de uicación

In [157]:
# Averiguando en Chatgpt veo la función para devolver la dirección con las coordenadas
# para eso tomo una fila y hago el ejemplo

In [18]:
def obtener_direccion(COORDENADAS):
    latitud, longitud = COORDENADAS
    geolocalizador = Nominatim(user_agent="mi_aplicacion")
    ubicacion = geolocalizador.reverse(COORDENADAS, language='es')
    return ubicacion.address

In [19]:
# tomo ejemplo de Coordenadas geográficas, suponiendo que posx es latitud y posy altitud
LATITUD = -34.68757022
LONGITUD = -58.47533969

In [23]:
obtener_direccion((-34.68757022, -58.47533969))

'6502, Avenida General Francisco Fernández de La Cruz, Villa Riachuelo, Buenos Aires, Distrito del Deporte, Comuna 8, Ciudad Autónoma de Buenos Aires, C1439COV, Argentina'

In [78]:
# En realidad funcionó!!!, la dirección que me da el sistema es
# Dirección: 6502, Avenida General Francisco Fernández de La Cruz, Villa Riachuelo, Buenos Aires, Distrito del Deporte, Comuna 8, Ciudad Autónoma de Buenos Aires, C1439COV, Argentina
# Tambien cuadra numero de la comuna

In [24]:
# ya sabiendo que si son las coordenadas, Cambio el nombre de las columnas por latitud y longitud, las junto en una que se 
# llame coordenadas y con esa columna ya tendría toda la información

df_hechos = df_hechos.rename(columns={'pos x': 'LONGITUD'})
df_hechos = df_hechos.rename(columns={'pos y': 'LATITUD'})


In [ ]:
df_hechos.head()

In [27]:
df_hechos.size

11832

In [28]:
# Junto las 2 columnas en una que se llame COORDENADAS, por que la función que encontré solo tiene un argumento

df_hechos['COORDENADAS'] = df_hechos.apply(lambda row: (row['LONGITUD'], row['LATITUD']), axis=1)


In [29]:
# con la columna coordenadas tengo toda la descripción de la ubicación, es importante comuna, 
# Me aseguro que las columnas altitud y latitud sean numericas

df_hechos['ALTITUD'] = df_hechos['LATITUD'].apply(pd.to_numeric, errors='coerce')

df_hechos['LONGITUD'] = df_hechos['LONGITUD'].apply(pd.to_numeric, errors='coerce')


In [ ]:
df_hechos.head()

In [30]:
# Con las coordenada ya tengo toda la info y puedo graficar en powebi asi que borro las que no interesan tanto

Columnas_borrar = ['TIPO_DE_CALLE', 'Calle', 'Altura',  'XY (CABA)', 'LONGITUD', 'ALTITUD',	'LUGAR_DEL_HECHO', 'LATITUD', 'PARTICIPANTES']


In [31]:
df_hechos =  df_hechos.drop(Columnas_borrar, axis=1)


In [ ]:
df_hechos.head()

In [33]:
# Voy a renombrar Cruce y Dirección Normalizada por que todo esta en mayusculas asi que no se ve bien

df_hechos = df_hechos.rename(columns={'Cruce': 'CRUCE'})
df_hechos = df_hechos.rename(columns={'Dirección Normalizada': 'DIRECCION'})


In [ ]:
df_hechos.head()

In [35]:
# Desarrollo función para verificar datos en el df


def verificar_tipo_datos(df):
# Genero Diccionario vació para almacenar información sobre cada columna
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}
    
# Iterar sobre  las columnas del DataFrame df
    for columna in df.columns:
        # Calcular el porcentaje de valores no nulos en la columna actual
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        
# Agregar información de mi_dict
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100 - porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())
        
# Crear un DataFrame con la información recopilada
    df_info = pd.DataFrame(mi_dict)
    
# Calcular el total de valores nulos en el DataFrame
    total_nulos = df.isnull().sum().sum()
    
# Imprimir el total de valores nulos
    print(f'Total de valores nulos en el DataFrame: {total_nulos}')
    
# Mostrar las columnas con valores nulos y sus respectivos porcentajes (ordenadas de mayor a menor)
    columnas_con_nulos = df_info[df_info["nulos"] > 0][["nombre_campo", "nulos_%"]].sort_values(by="nulos_%", ascending=False)
    if not columnas_con_nulos.empty:
        print("\nColumnas con valores nulos y sus porcentajes:")
        print(columnas_con_nulos)
        
# Devolver el DataFrame con la información de cada columna

    return df_info

In [36]:
verificar_tipo_datos(df_hechos)

Total de valores nulos en el DataFrame: 179

Columnas con valores nulos y sus porcentajes:
  nombre_campo  nulos_%
4        CRUCE    24.57
5    DIRECCION     1.15


,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,ID,[<class 'str'>],100.00,0.00,0
1,N_VICTIMAS,[<class 'int'>],100.00,0.00,0
2,FECHA,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.00,0.00,0
3,HORA,[<class 'str'>],100.00,0.00,0
4,CRUCE,"[<class 'str'>, <class 'float'>]",75.43,24.57,171
5,DIRECCION,"[<class 'str'>, <class 'float'>]",98.85,1.15,8
6,COMUNA,[<class 'int'>],100.00,0.00,0
7,VICTIMA,[<class 'str'>],100.00,0.00,0
8,ACUSADO,[<class 'str'>],100.00,0.00,0
9,COORDENADAS,[<class 'tuple'>],100.00,0.00,0


In [ ]:
# En esta caracterización de datos veo que hay 24% de nulos en cruce, yo creo que puedo borrar la 
# columna por que no me genera mayor información pero sime quitaría mucha si los borro
# El la columna dirección borrare los nulos por que solo es el 1.15%

In [37]:
# Borro nulos en dirección

df_hechos = df_hechos.dropna(subset=['DIRECCION'])


In [40]:
# Borro columna cruce

df_hechos = df_hechos.drop(columns=['CRUCE'])


In [ ]:
# Verifico nuevamente los datos del dataset

In [41]:
verificar_tipo_datos(df_hechos)

Total de valores nulos en el DataFrame: 0


,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,ID,[<class 'str'>],100.0,0.0,0
1,N_VICTIMAS,[<class 'int'>],100.0,0.0,0
2,FECHA,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.0,0.0,0
3,HORA,[<class 'str'>],100.0,0.0,0
4,DIRECCION,[<class 'str'>],100.0,0.0,0
5,COMUNA,[<class 'int'>],100.0,0.0,0
6,VICTIMA,[<class 'str'>],100.0,0.0,0
7,ACUSADO,[<class 'str'>],100.0,0.0,0
8,COORDENADAS,[<class 'tuple'>],100.0,0.0,0


In [ ]:
# Confirmo que los datos están listos para ser analizados en un eda, grabo el dataframe df_hechos

In [42]:
df_hechos.to_parquet('df_hechos.parquet')
